# Análisis de los sitios de lanzamiento con Folium 🗺️

<strong>Objetivos:</strong>
+ Marcar todos los sitios de lanzamiento en un mapa.
+ Marcar los lanzamientos exitosos/fallidos para cada sitio en el mapa.
+ Calcular las distancias entre un sitio de lanzamiento y sus proximidades.

Después de cumpletar los objetivos, debería poder encontrar algunos patrones geográficos.

## Importar librerías y cargar datos

In [5]:
# Importar.
import folium
import wget
import pandas as pd
from math import sin, cos, sqrt, atan2, radians

# Importar plugins de Folium.
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

# Para ignorar Warnings.
import warnings
warnings.filterwarnings('ignore')

El siguiente dataset con el nombre <code>spacex_launch_geo.csv</code> es un dataset aumentado, con latitud y longitud agregadas para cada sitio de lanzamiento.

In [7]:
# Descargar y cargar datos.
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv', '../datasets')
spacex_df = pd.read_csv(spacex_csv_file)

In [8]:
# Crear un DataFrame seleccionando las sub-columnas: 'Launch Site', 'Lat(Latitude)', 'Long(Longitude)', 'class'.
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


Las coordenadas anteriores son solo números simples que no pueden brindar información intuitiva sobre dónde están esos sitios de lanzamiento. Es necesario entonces visualizar las coordenadas fijándolas en un mapa.

## Tareas

### Tarea 1: Marcar todos los sitios de lanzamiento en un mapa

Primero, crear un objeto <code>Map</code> de Folium, con una ubicación central inicial: la NASA Johnson Space Center, en Houston, Texas.

In [9]:
nasa_coordinate = [29.559684888503615, -95.0830971930759] # <-- Coordenadas de la ubicación inicial.
site_map = folium.Map(location=nasa_coordinate, zoom_start=10) # <-- Objeto Map.

Usar <code>folium.Circle</code> para agregar un área circular, resaltada con una etiqueta de texto en una coordenada específica.<br> 
Ejemplo:

In [12]:
# Crear un círculo para las coordenadas que dan a la NASA Johnson Space Center's,
# con una etiqueta emergente que muestre su nombre.
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size = (20, 20),
        icon_anchor = (0, 0),
        html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Debería poder verse un pequeño círculo naranja cerca de la ciudad de Houston y poder acercar para ver un círculo más grande.

Ahora, agregar un círculo para cada sitio de lanzamiento en el DataFrame <code>launch_sites_df</code>.

In [13]:
'''
Crear y agregar folium.Circle y folium.Marker para marcar cada sitio de lanzamiento en el mapa.
'''
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
for lat, lng, label in zip(launch_sites_df['Lat'], launch_sites_df['Long'], launch_sites_df['Launch Site']):
    coordinate = [lat, lng]
    circle = folium.Circle(coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(label))
    marker = folium.map.Marker(
        coordinate, 
        icon = DivIcon(
            icon_size = (20, 20),
            icon_anchor = (0, 0), 
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % label, 
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

Ahora, explorar el mapa acercando o alejando las áreas marcadas e intentar responder las siguientes preguntas:

+ ¿Todos los sitios de lanzamiento están cerca de la línea del Ecuador?
+ ¿Están los los sitios de lanzamiento muy cerca de la costa?
  
También tratar de explicar algunos hallazgos relevantes.

### Tarea 2: Marcar los lanzamientos exitosos/fallidos para cada sitio en el mapa

Intentar mejorar el mapa agregando los resultados de lanzamiento para cada sitio y ver qué sitios tienen altas tasas de éxito. Recordar que el DataFrame spacex_df tiene registros de lanzamiento detallados, y que la columna <code>class</code> indica si este lanzamiento fue exitoso o no (si aterrizó con éxito o no).

A continuación, crear marcadores para todos los registros de lanzamiento. Si un lanzamiento fue exitoso (clase=1), entonces usar un marcador verde y si un lanzamiento falló, usar un marcador rojo (clase=0)

Tener en cuenta que un lanzamiento solo ocurre en uno de los cuatro sitios de lanzamiento, lo que significa que muchos registros de lanzamiento tendrán exactamente la misma coordenada. Los grupos de marcadores pueden ser una buena manera de simplificar un mapa que contiene muchos marcadores que tienen la misma coordenada.

In [14]:
# Primero, crear un objeto MarkerCluster.
marker_cluster = MarkerCluster()

In [15]:
'''
Crear una nueva columna en el DataFrame launch_sites llamada 'marker_color' para almacenar los colores del marcador según el valor de la clase.
'''
def func(item):
    if item == 1:
        return 'green'
    else:
        return 'red'

launch_sites_df['marker_color'] = launch_sites_df['class'].apply(func)
launch_sites_df

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS SLC-40,28.563197,-80.576820,1,green
2,KSC LC-39A,28.573255,-80.646895,1,green
3,VAFB SLC-4E,34.632834,-120.610745,0,red


In [16]:
'''
Crear una nueva columna en el DataFrame spacex_df llamada 'marker_color' para almacenar los colores del marcador según el valor de la clase.
'''
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


Para cada resultado de lanzamiento en el DataFrame spacex_df, agregar <code>folium.Marker</code> a <code>marker_cluster</code>.

In [17]:
# Agregar marker_cluster a site_map.
site_map.add_child(marker_cluster)

'''
Para cada fila en spacex_df,
crear un objeto Marker con su coordenada
y personalizar el icono del marcador para indicar si este lanzamiento fue exitoso o falló.
'''
for lat, lng, label, color in zip(spacex_df['Lat'], spacex_df['Long'], spacex_df['Launch Site'], spacex_df['marker_color']):
    coordinate = [lat, lng]
    marker = folium.Marker(
        coordinate, 
        icon = folium.Icon(color='white', icon_color=color),
        popup = label
    )
    marker_cluster.add_child(marker)

site_map

Acercar para ver mejor.<br>

A partir de los marcadores con etiquetas de color, se debería poder identificar fácilmente qué sitios de lanzamiento tienen tasas de éxito relativamente altas.

### Tarea 3: Calcular las distancias entre un sitio de lanzamiento y sus proximidades

A continuación, explorar y analizar las proximidades de los sitios de lanzamiento.

Primero agregar <code>MousePosition</code> en el mapa para obtener coordenadas para un mouse sobre un punto en el mapa. Así, mientras se explora el mapa, se pueden encontrar fácilmente las coordenadas de cualquier punto de interés.

In [18]:
# Añadir Mouse Position para obtener las coordenadas (Lat, Long) para un mouse sobre el mapa.
formatter = 'function(num) {return L.Util.formatNum(num, 5);};'
mouse_position = MousePosition(
    position = 'topright',
    separator = 'Long:',
    empty_string = 'NaN',
    lng_first = False,
    num_digits = 20,
    prefix = 'Lat:',
    lat_formatter = formatter,
    lng_formatter = formatter,
)

site_map.add_child(mouse_position)
site_map

Ahora acercarse a un sitio de lanzamiento y explorar su proximidad para ver si se puede encontrar fácilmente alguna vía férrea, autopista, línea costera, etc. Mover el mouse a estos puntos y anotar sus coordenadas (que se muestran en la parte superior izquierda).

Se puede calcular la distancia entre dos puntos en el mapa en función de sus valores de latitud y longitud, utilizando el siguiente método:

In [19]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # Radio aproximado de la tierra en km.
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Marcar un punto en la costa más cercana con <code>MousePosition</code> y calcular la distancia entre el punto de la costa y el lugar de lanzamiento.

In [20]:
# Encontrar las coordenadas de la costa más cercana.
# Ej: Lat: 28.56367, Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
coastline_lat = 28.56398
coastline_lon = -80.56809
launch_site_lat = 28.56321
launch_site_lon = -80.57683

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

Después de obtener las coordenadas, crear un <code>folium.Marker</code> para mostrar la distancia.

In [21]:
# Crear y añadir folium.Marker en el punto de la costa más cercano seleccionado en el mapa.
# Mostrar la distancia entre el punto de la costa y el sitio de lanzamiento usando la propiedad del icono.
'''
Por ejemplo:
distance_marker = folium.Marker(
   coordinate,
   icon = DivIcon(
       icon_size=(20, 20),
       icon_anchor=(0, 0),
       html = '<div style = 'font-size:12; color:#d35400;'><b>%s</b></div>' % '{:10.2f} KM'.format(distance),
       )
   )
'''
coast_coordinates = [coastline_lat, coastline_lon]
distance_marker = folium.Marker(
    coast_coordinates,
    icon = DivIcon(
        icon_size = (20, 20),
        icon_anchor = (0, 0),
        html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
distance_marker.add_to(site_map)
site_map

Dibujar un <code>PolyLine</code> entre un sitio de lanzamiento y el punto de costa seleccionado.

In [22]:
# lines = folium.PolyLine(locations=coordinates, weight=1)
launch_site_coordinates = [launch_site_lat, launch_site_lon]
lines = folium.PolyLine(locations=[coast_coordinates, launch_site_coordinates], weight=1)
site_map.add_child(lines)

Del mismo modo, se puede trazar una línea entre un sitio de lanzamiento y su ciudad, costa, autopista, etc. más cercana.

Crear un marcador con la distancia a la ciudad, vía férrea, autopista, etc. más cercana.

Dibujar una línea entre el marcador y el sitio de lanzamiento.

In [23]:
# Ciudad más cercana: Titusville.
# Dibujar una línea entre la ciudad Titusville y el sitio de lanzamiento CCAFS SLC-40.
city_lat = 28.61208
city_lon = -80.80764
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

city_coordinates = [city_lat, city_lon]
distance_marker = folium.Marker(
    city_coordinates,
    icon = DivIcon(
        icon_size = (20, 20),
        icon_anchor = (0, 0),
        html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
        )
    )
distance_marker.add_to(site_map)

launch_site_coordinates = [launch_site_lat, launch_site_lon]
lines = folium.PolyLine(locations=[city_coordinates, launch_site_coordinates], weight=1)
site_map.add_child(lines)
site_map

Acercar para ver la línea azul.

In [24]:
# Ferrocarril más cercano: NASA railroad.
# Dibujar una línea entre el ferrocarril NASA railroad y el sitio de lanzamiento CCAFS SLC-40.
railway_lat = 28.57208
railway_lon = -80.58527
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

railway_coordinates = [railway_lat, railway_lon]
distance_marker = folium.Marker(
    railway_coordinates,
    icon = DivIcon(
        icon_size = (20, 20),
        icon_anchor = (0, 0),
        html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
        )
    )
distance_marker.add_to(site_map)

launch_site_coordinates = [launch_site_lat, launch_site_lon]
lines = folium.PolyLine(locations=[railway_coordinates, launch_site_coordinates], weight=1)
site_map.add_child(lines)
site_map

Acercar para ver la línea azul hacía el ferrocarril.

In [25]:
# Dibujar una línea entre la carretera más cercana y el sitio de lanzamiento CCAFS SLC-40.
highway_lat = 28.56478
highway_lon = -80.57103
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

highway_coordinates = [highway_lat, highway_lon]
distance_marker = folium.Marker(
    highway_coordinates,
    icon = DivIcon(
        icon_size = (20, 20),
        icon_anchor = (0, 0),
        html = '<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
        )
    )
distance_marker.add_to(site_map)

launch_site_coordinates = [launch_site_lat, launch_site_lon]
lines = folium.PolyLine(locations=[highway_coordinates, launch_site_coordinates], weight=1)
site_map.add_child(lines)
site_map

Acercar para ver la línea azul hacia la carretera.

Después de trazar las líneas de la distancia a las proximidades, se puede responder fácilmente a las siguientes preguntas:
+ ¿Los sitios de lanzamiento están muy cerca de los ferrocarriles?
+ ¿Los sitios de lanzamiento están muy cerca de las carreteras?
+ ¿Los sitios de lanzamiento están muy cerca de la costa?
+ ¿Los sitios de lanzamiento mantienen cierta distancia de las ciudades?


También tratar de explicar los hallazgos.

Se han descubierto muchas cosas interesantes relacionadas con la ubicación de los sitios de lanzamiento usando Folium, de una manera muy interactiva. En el siguiente cuaderno, tendrá lugar la creación de un tablero usando Ploty Dash.

## Resumen

El desarrollo de este cuaderno puede resumirse en los siguientes puntos:

+ El primer paso fue Importar las librerías necesarias, incluyendo plugins de Folium.
+ Luego se descargó <code>spacex_launch_geo.csv</code>, que es un dataset aumentado, con latitud y longitud agregadas para cada sitio de lanzamiento. Se cargó este dataset como un DataFrame llamado <code>spacex_df</code>, a partir del cual se obtuvo el DataFrame final <code>launch_sites_df</code> solo con las columnas <code>Launch Site</code>, <code>Lat</code>, <code>Long</code> y <code>class</code>.
+ Se creó un objeto <code>Map</code> de Folium para visualizar las coordenadas del dataset, esto fijándolas en un mapa. Se determinó como ubicación central inicial la NASA Johnson Space Center, en Houston, Texas. Además, se usó <code>folium.Circle</code> para crear un círculo para las coordenadas que dan a la NASA Johnson Space Center's, junto con una etiqueta emergente que muestra su nombre.
+ Luego se creó y agregó un círculo y etiqueta emergente para cada sitio de lanzamiento en el mapa, esto usando el DataFrame <code>launch_sites_df</code>.
+ Tambíen se marcaron los lanzamientos exitosos/fallidos para cada sitio en el mapa y se calcularon las distancias entre un sitio de lanzamiento y sus proximidades.
+ Se completaron todas las tareas.

---